In [1]:
import os
import re
import tqdm
import string
import pandas as pd
import numpy as np
import util
from sklearn.decomposition import TruncatedSVD
from data_preprocessing import *
from word_embedding_load import *
from classification import *
from xgboost_classifier import *
from testaccuracy import *

C:\Users\zhuya\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.
C:\Users\zhuya\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
[all_data, train_size, test_size, train_x, train_y, test_x] = util.loadData()

In [3]:
sentences = data_preprocess(all_data)

In [4]:
Text_INPUT_DIM=200
param = util.Doc2VecParam(1, 2, 200, 1e-4, 5, 4, 30, 1)
filename='../model/doc2vec/docEmbeddings_win2_load_all.d2v'


In [5]:
GENE_INPUT_DIM=25
TEXT_INPUT_DIM=200
svd = TruncatedSVD(n_components=25, n_iter=GENE_INPUT_DIM, random_state=12)

In [6]:
text_model = getTextModel(sentences, param, filename, source_file = "../data/bio_nlp_vec/PubMed-shuffle-win-2.bin")
truncated_one_hot_gene = getGeneVec(all_data, svd)
truncated_one_hot_variation = getVariationVec(all_data, svd)
text_train_arrays, text_test_arrays = getTextVec(text_model, train_size, test_size, TEXT_INPUT_DIM)

Creating model...
successfully created the Text Model and save it to ../model/doc2vec/docEmbeddings_win2_load_all.d2v


In [7]:
print(text_train_arrays.shape)
print(text_test_arrays.shape)
text_train_arrays[0]

(3689, 200)
(986, 200)


array([ 1.9926796 , -0.95616704, -0.41943806,  1.05640149, -0.17750672,
       -0.22473556, -1.14849627, -1.04512095,  0.72278821, -0.26906219,
       -0.12173071,  0.27951074,  0.85554093,  1.4066658 , -0.66658169,
       -3.5733881 ,  0.61995304, -2.23058319, -0.13402276,  2.12668133,
       -0.35180882, -0.27377269,  0.54994392, -1.48118317, -1.4935751 ,
       -1.68159318,  0.34006077,  2.6541965 , -0.45972574, -0.43201742,
        0.3170231 , -1.63518965,  2.0632391 ,  1.46774471, -0.95080161,
        0.16261423,  1.81780541,  0.94335526, -0.02719352,  0.66872084,
        0.53617448,  0.2284466 ,  1.14089978, -1.53023636, -0.30838454,
        1.86624837, -0.64276832,  0.49743181, -0.53058469,  1.06889701,
        0.34412953,  0.14060664,  1.23631251,  0.41249499, -0.50394237,
        0.92049599,  0.2226413 , -0.11276353, -0.56527591,  0.82244128,
       -0.99196661, -0.42952666, -0.88652813,  0.43424979, -1.98911691,
        1.09482539, -2.45888305,  1.73302603,  2.20067   , -0.60

In [8]:
train_set = np.hstack((truncated_one_hot_gene[:train_size], truncated_one_hot_variation[:train_size], text_train_arrays))
test_set = np.hstack((truncated_one_hot_gene[train_size:], truncated_one_hot_variation[train_size:], text_test_arrays))
encoded_y = pd.get_dummies(train_y)
encoded_y = np.array(encoded_y)

In [9]:
print(train_set.shape)
print(test_set.shape)

(3689, 250)
(986, 250)


In [10]:
train_set[0, 25:50]

array([  1.00000000e+00,   4.18523246e-19,   1.82140582e-19,
         5.52932734e-29,   9.01317672e-25,   1.47967745e-22,
        -8.38240178e-21,   2.00300590e-21,   9.74158370e-21,
        -9.34683441e-21,   1.91083889e-20,   6.55003880e-19,
        -1.09276438e-17,  -1.56730196e-18,  -2.12913704e-18,
        -4.51096312e-18,  -3.82900024e-19,   1.15894157e-17,
         1.59938703e-17,   3.58735292e-17,  -3.35304994e-18,
         1.68462844e-17,   6.76851697e-18,   5.86047343e-17,
         7.80129580e-21])

In [11]:
encoded_y.shape

(3689, 9)

In [14]:
# this is the 4 layer full-connected nerual network model
model = nn_baseline_model(TEXT_INPUT_DIM, GENE_INPUT_DIM, GENE_INPUT_DIM)
model.summary()

C:\Users\zhuya\Desktop\semester 1\249 data structure\project\src\classification.py:28: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, kernel_initializer="normal", input_dim=250, activation="relu")`
  model.add(Dense(256, input_dim=Text_INPUT_DIM+ Gene_INPUT_DIM + Variation_INPUT_DIM, init='normal', activation='relu'))
C:\Users\zhuya\Desktop\semester 1\249 data structure\project\src\classification.py:30: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, kernel_initializer="normal", activation="relu")`
  model.add(Dense(256, init='normal', activation='relu'))
C:\Users\zhuya\Desktop\semester 1\249 data structure\project\src\classification.py:32: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(80, kernel_initializer="normal", activation="relu")`
  model.add(Dense(80, init='normal', activation='relu'))
C:\Users\zhuya\Desktop\semester 1\249 data structure\project\src\classification.py:33: UserWarning: Update your `Dense` call to the

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 256)               64256     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 80)                20560     
_________________________________________________________________
dense_8 (Dense)              (None, 9)                 729       
Total params: 151,337
Trainable params: 151,337
Non-trainable params: 0
_________________________________________________________________


In [15]:
model = train_nn_model(model, train_set, encoded_y, filename = 'try(win2load).h5')
y_predict = model.predict_proba(test_set)

successful load

 32/986 [..............................] - ETA: 0s

In [20]:
#the xgboost classfication model
#first deal with the input label, transfrom it from 1-9 to 0-8(required by the xgboost)
for i in range(len(train_y)):
    train_y[i] -=1  

In [21]:
y_predict = xgbclassifier(train_set, train_y, test_set, 5, 10, 1000)

[0]	train-mlogloss:2.11096	valid-mlogloss:2.13755
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[50]	train-mlogloss:0.546776	valid-mlogloss:1.16003
[100]	train-mlogloss:0.200471	valid-mlogloss:0.985421
[150]	train-mlogloss:0.089525	valid-mlogloss:0.949457
[200]	train-mlogloss:0.047857	valid-mlogloss:0.95521
[250]	train-mlogloss:0.030732	valid-mlogloss:0.97078
Stopping. Best iteration:
[171]	train-mlogloss:0.067177	valid-mlogloss:0.948514

0.948514315681
1


In [22]:
savesubmisstion(y_predict, test_x, filename = "submission_all.csv")